In [1]:
# configure DuckDB to work with Iceberg and MinIO

import duckdb


duckdb.sql("""
INSTALL iceberg;
INSTALL httpfs;

UPDATE EXTENSIONS;

LOAD iceberg;
LOAD httpfs;

SET s3_endpoint='minio:9000';
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
SET s3_region='eu-central-1';
SET s3_use_ssl=false;
SET s3_url_style='path';
""")

In [2]:
# use pyiceberg to get the location of the table

from pyiceberg.catalog.rest import RestCatalog


catalog = RestCatalog(name="iceberg", uri="http://nessie:19120/iceberg/main/")
table = catalog.load_table("default.bank_transfers")

In [7]:
# query the table

duckdb.sql(f"SELECT * FROM iceberg_scan('{table.metadata_location}')")

┌───────┬────────┬──────────────────┬────────────────┬──────────────────────────┐
│  id   │ amount │ transferred_from │ transferred_to │        timestamp         │
│ int64 │ int32  │     varchar      │    varchar     │ timestamp with time zone │
├───────┼────────┼──────────────────┼────────────────┼──────────────────────────┤
│     4 │    200 │ CTX Inc.         │ XYZ GmbH       │ 2022-12-01 07:32:18+01   │
│     1 │  12000 │ ACME INC         │ ASTROCORP      │ 2022-11-14 01:55:00+01   │
│     2 │  24000 │ John Doe         │ Jane Doe       │ 2022-11-15 03:11:00+01   │
│     3 │    500 │ Deborah S.       │ Michael C.     │ 2022-11-17 17:25:07+01   │
└───────┴────────┴──────────────────┴────────────────┴──────────────────────────┘

In [12]:
# list snapshots of the table

duckdb.sql(f"SELECT * FROM iceberg_snapshots('{table.metadata_location}')")

┌─────────────────┬─────────────────────┬─────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ sequence_number │     snapshot_id     │      timestamp_ms       │                                                                          manifest_list                                                                           │
│     uint64      │       uint64        │        timestamp        │                                                                             varchar                                                                              │
├─────────────────┼─────────────────────┼─────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│               2 │ 7083951985285040312 │ 2025-03-15 20:30:13.031 │ s3://war